In [13]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

class AttendanceSystem:
    def __init__(self):
        self.face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False,
                                                         max_num_faces=1,
                                                         refine_landmarks=True,
                                                         min_detection_confidence=0.9,
                                                         min_tracking_confidence=0.9)
        self.drawer = mp.solutions.drawing_utils
        self.style = mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style()
        self.label_encoder = LabelEncoder()
        self.model = None

    def _get_camera(self):
        return cv2.VideoCapture(4)

    def _normalize_landmarks(self, landmarks_flat):
        landmarks = np.array(landmarks_flat).reshape(478, 3)
        centered = landmarks - landmarks[0]
        distance = np.linalg.norm(landmarks[33] - landmarks[263])
        normalized = centered / distance
        return normalized.flatten()

    def register_faces(self):
        vid = self._get_camera()
        all_faces = []
        num_of_persons = int(input('How many people do you want to register? '))
        for i in range(1, num_of_persons + 1):
            name = input(f"Enter the name for Person {i}: ").strip()
            samples = 0
            last_time = 0
            print(f"Please show your face, {name}. Capturing 300 samples...")

            while samples < 300:
                b, frame = vid.read()
                if not b:
                    print("Frame not captured.")
                    break

                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                output = self.face_mesh.process(rgb)

                if output.multi_face_landmarks:
                    landmarks = output.multi_face_landmarks[0]
                    self.drawer.draw_landmarks(frame, landmarks, mp.solutions.face_mesh.FACEMESH_TESSELATION, None, self.style)
                    now = time.time()
                    if now - last_time > 0.02:
                        face = []
                        for lm in landmarks.landmark:
                            face.extend([lm.x, lm.y, lm.z])
                        face.append(name)
                        all_faces.append(face)
                        samples += 1
                        last_time = now

                cv2.putText(frame, f"{name}: {samples}/300", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.imshow("Registering", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        print(f'{num_of_persons} person(s) registered successfully.')
        vid.release()
        cv2.destroyAllWindows()

        # Data preparation
        final_data = pd.DataFrame(all_faces)
        final_data.dropna(inplace=True)

        x1 = final_data.iloc[:, :-1]
        y1 = final_data.iloc[:, -1]
        x_train = [self._normalize_landmarks(row) for row in x1.values]
        y_train = self.label_encoder.fit_transform(y1)

        # Model training
        print("Training model now...")
        rfc_model = RandomForestClassifier(n_estimators=66,
                                           max_depth=15,
                                           min_samples_split=2,
                                           min_samples_leaf=1,
                                           random_state=42)
        rfc_model.fit(x_train, y_train)
        self.model = rfc_model
        print("Model training complete.")

    def mark_attendance(self):
        if self.model is None:
            print("Model is not trained yet. Please register faces first.")
            return

        print("Look into the camera to mark your attendance.")
        load_model = self.model
        label_enc = self.label_encoder
        vid = self._get_camera()
        attendance_log = {}
        last_seen = {}
        cooldown = 5  # seconds

        while True:
            ret, frame = vid.read()
            if not ret:
                break

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            output = self.face_mesh.process(rgb)

            if output.multi_face_landmarks:
                landmarks = output.multi_face_landmarks[0]
                self.drawer.draw_landmarks(frame, landmarks, mp.solutions.face_mesh.FACEMESH_TESSELATION, None, self.style)

                face = []
                for lm in landmarks.landmark:
                    face.extend([lm.x, lm.y, lm.z])
                x_test = [self._normalize_landmarks(face)]
                pred = load_model.predict(x_test)[0]
                name = label_enc.inverse_transform([pred])[0]
                now = time.strftime('%H:%M:%S')

                if name not in last_seen or (time.time() - last_seen[name]) > cooldown:
                    if name not in attendance_log:
                        attendance_log[name] = {'login': now, 'logout': None}
                        print(f"{name} logged in at {now}")
                    elif attendance_log[name]['logout'] is None:
                        attendance_log[name]['logout'] = now
                        print(f"{name} logged out at {now}")
                    last_seen[name] = time.time()

                cv2.putText(frame, name, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

            cv2.imshow("Attendance Tracking", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        vid.release()
        cv2.destroyAllWindows()

        # Store attendance
        self.data_frame = pd.DataFrame([
            {'Name': name, 'InTime': data['login'], 'OutTime': data['logout']}
            for name, data in attendance_log.items()
        ])
        print("Attendance has been marked successfully.")

    def get_attendance_report(self):
        if self.data_frame is None:
            print("No attendance recorded.")
            return

        df = self.data_frame
        try:
            df['InTime'] = pd.to_datetime(df['InTime'], format='%H:%M:%S')
            df['OutTime'] = pd.to_datetime(df['OutTime'], format='%H:%M:%S')
            df['Working_Hours'] = df['OutTime'] - df['InTime']
            expected_start_time = pd.to_datetime('09:00:00', format='%H:%M:%S')
            df['LoginStatus'] = df['InTime'].apply(lambda x: 'Late Login' if x > expected_start_time else 'On Time')
            df['LoginTime'] = df['InTime'].dt.strftime('%H:%M:%S')
            df['LogoutTime'] = df['OutTime'].dt.strftime('%H:%M:%S')
            df1 = df[['Name', 'LoginTime', 'LoginStatus', 'LogoutTime', 'Working_Hours']]

            print("\nAttendance Report:")
            for _, row in df1.iterrows():
                print(f"{row['Name']}, {row['LoginTime']}, {row['LoginStatus']}, {row['LogoutTime']}, {row['Working_Hours']}")

            save_choice = input("Do you want to save this report as CSV? (y/n): ").strip().lower()
            if save_choice == 'y':
                filename = f"attendance_report_{time.strftime('%Y%m%d_%H%M%S')}.csv"
                df1.to_csv(filename, index=False)
                print(f"Report saved successfully as '{filename}'")
        except Exception as e:
            print(f"Failed to process attendance report: {e}")

if __name__ == "__main__":
    system = AttendanceSystem()
    while True:
        print("\n--- Face Attendance System ---")
        print("1. Register and Train a User")
        print("2. Give Attendance")
        print("3. Today's Attendance sheet")
        print("4. Exit")
        choice = input("Enter your choice (1/2/3/4): ").strip()

        if choice == '1':
            system.register_faces()
        elif choice == '2':
            system.mark_attendance()
        elif choice == '3':
            system.get_attendance_report()
        elif choice == '4':
            print("Exiting Attendance System.")
            break
        else:
            print("Invalid choice. Please try again.")


--- Face Attendance System ---
1. Register and Train a New User
2. Give Attendance
3. Today's Attendance sheet
4. Exit


Enter your choice (1/2/3/4):  1
How many people do you want to register?  1
Enter the name for Person 1:  kiran


Please show your face, kiran. Capturing 300 samples...
1 person(s) registered successfully.
Training model now...
Model training complete.

--- Face Attendance System ---
1. Register and Train a New User
2. Give Attendance
3. Today's Attendance sheet
4. Exit


Enter your choice (1/2/3/4):  2


Look into the camera to mark your attendance.
kiran logged in at 12:00:41
kiran logged out at 12:00:52
Attendance has been marked successfully.

--- Face Attendance System ---
1. Register and Train a New User
2. Give Attendance
3. Today's Attendance sheet
4. Exit


...


Attendance Report:
kiran, 12:00:41, Late Login, 12:00:52, 0 days 00:00:11


Do you want to save this report as CSV? (y/n):  y


Report saved successfully as 'attendance_report_20250717_120104.csv'

--- Face Attendance System ---
1. Register and Train a New User
2. Give Attendance
3. Today's Attendance sheet
4. Exit


Enter your choice (1/2/3/4):  4


Exiting Attendance System.
